In [1]:
# Some tests of NuSTAR FoV map in comparison to other X-ray data
# 
# 11-Apr-2023   IGH
# 12-Apr-2023   Fixed nustar_pysolar maps to work with older evt converted files
# 17-Apr-2023   Add in label per NuSTAR detector quadrant
#               Added in more events
# 21-Apr-2023   Added in AIA HEK flares
# 30-Jan-2024   Reran for Dec 2023 data
# 15-May-2024   Reran for Mar 2024 data
# 12-Jul-2024   Reran for Jun 2024 data
# 06-Aug-2024   Reran for Jul 2024 data

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from sunpy.net import Fido
from sunpy.net import attrs as a
from sunpy import timeseries as ts
from sunpy.time import parse_time
from sunpy.coordinates import frames, get_horizons_coord
import sunpy.map
from astropy.coordinates import SkyCoord
from astropy.io import fits
import astropy.units as u
import datetime
import pandas as pd
import os
from pathlib import Path
import glob
from stixdcpy.quicklook import LightCurves

import nustar_pysolar as nustar
from scipy import ndimage

import matplotlib.colors as colors
from matplotlib.colors import LogNorm
from sunpy.coordinates import Helioprojective

import nsx_func

import warnings
warnings.simplefilter('ignore')

plt.rcParams.update({'font.size': 20,'font.family':"sans-serif",\
            'font.sans-serif':"Arial",'mathtext.default':"regular"})

In [3]:
# List of all the NuSTAR observations, with my naming, 
# based off first day of observing per campaign
dobs=['20140910','20141101','20141211',
    '20150429','20150901',
    '20160219','20160422','20160726',
    '20170321','20170821','20170911','20171010',
    '20180529','20180907','20180928',
    '20190112','20190412','20190425','20190702',
    '20200129','20200221','20200606','20200912',
    '20210108','20210429','20210720','20210730','20211117',
    '20220224','20220603','20220906','20221209',
    '20230318','20231228',
    '20240330','20240630','20240716']

# Data directory
# ddir=str(Path.home())+'/data/heasarc_nustar/'
ddir='/Volumes/Samsung_T5/data/heasarc_nustar/'

In [4]:
# # Find all pointings/IDs for the chosen observation 
# for icamp in [21,23,24,25,26,27,28,29,30,31,32]:
# # for icamp in [29]:

#     maindir=ddir+'ns_'+dobs[icamp]+'/'
#     print(icamp, ' --- ',maindir)
#     # Most start with 20 or 80 or 90?
#     ids = [f.name for f in os.scandir(maindir) \
#         if (f.is_dir() and (f.name.startswith('20') or f.name.startswith('80') or f.name.startswith('90')))]
#     ids=sorted(ids)

#     # Filter out random other dirs
#     ids=[f for f in ids if not f.endswith('_new')]

#     # Remove the mosiac from 20220603
#     if (dobs[icamp] == '20220603'):
#         ids=[f for f in ids if f.startswith('2062400')]
        
#     print("#: ",len(ids),' --- ',ids)